In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
# Environment setup
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_DASH"]
init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs)

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



2400 intervals, or 50 days of data at 30 minutes period downloaded.


In [ ]:
# Training run
# Training params
nb_steps = 500
batch_size = 2
nb_max_episode_steps = 7

agent = apriori.MomentumTrader()

hp = {
    'ma1': [2, env.obs_steps],
    'ma2': [2, env.obs_steps],
    'std_span': [2, env.obs_steps],
    'alpha_up': [1e-4, 5e-1],
    'alpha_down': [1e-4, 5e-1]
    }

search_space = {'mean_type':{'simple': hp,
                             'exp': hp,
                             'kama': hp
                             }
                }
constrains = [lambda mean_type, ma1, ma2, std_span, alpha_up, alpha_down: ma1 < ma2]

opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, constrains, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

# Previous found params:
# {'mean_type': 'kama', 'alpha_down': 0.4128425807825884, 'alpha_up': 0.37140222586733046, 'ma1': 133, 'ma2': 234, 'std_span': 39}

agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 500 steps with batch size 2...
Optimization step 410/500, step reward: 1.0008834279053551, ETC: 0 days 00:04:35.108428                     
 {'mean_type': 'simple', 'alpha_down': 0.1324423623544896, 'alpha_up': 0.17832492793466043, 'ma1': 44.87571054163683, 'ma2': 189.96451722420966, 'std_span': 214.26699311744932} 
 {'OOD': False, 'Error': False, 'ValueError': False, 'ActionError': False}
>> step 2099/2099, 100 % done, Cumulative Reward: 0.9151568554815869, ETC: 0 days 00:00:00                                 
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 0.000664
Portifolio / Benchmark Sharpe ratio:      -0.181036 / -0.128948
Portifolio / Benchmark Omega ratio:       0.964928 / 0.976824
Portifolio / Benchmark max drawdown:      -1.601913 / -246.913594


In [ ]:
# Validation run
tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))

agent.test(env, verbose=True)
env.plot_results();

2400 intervals, or 50 days of data at 30 minutes period downloaded.


In [ ]:
# Test run
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=50)))

agent.test(env, verbose=True)
env.plot_results();